<h1 align="center">
  Nienadzorowana reprezentacja autoenkodery i modele generatywne
</h1>

<h4 align="center">
  11.10.2023
</h4>
<br/>


## Mieszanina rozkładów normalnych i estymacja gęstości

### Mieszanina rozkładów (GMM)

Zobacz <a href="https://en.wikipedia.org/wiki/Mixture_model">tutaj</a>. Klasa <a href="http://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html#sklearn.mixture.GaussianMixture">GaussianMixture</a> dostarcza nam narzędzi, którymi możemy opisywać dane jako mieszaninę rozkładów normalnych.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import mixture

n = 200
mean = [-4, -5]
cov = [[1, 2], [2, 5]]
X = np.random.multivariate_normal(mean, cov, n)
y = -np.ones(n)

n = 300
mean = [3, 3]
cov = [[1, 0], [0, 1]]
X = np.concatenate((X, np.random.multivariate_normal(mean, cov, n)), axis=0)
y = np.concatenate((y, np.ones(n)), axis=0) 

n = 250
mean = [2, -7]
cov = [[1, 0], [0, 1]]
X = np.concatenate((X, np.random.multivariate_normal(mean, cov, n)), axis=0)
y = np.concatenate((y, 2*np.ones(n)), axis=0) 

permut = np.random.permutation(y.size)

X = X[permut, :]
labels_true = y[permut]

#----------------------------------------------------------------------
# Visualize the clustering
def plot_clustering(X, labels, title=None):

    plt.figure(figsize=(6, 6))
    plt.scatter(X[:, 0], X[:, 1], c=labels)
    
    plt.xticks([])
    plt.yticks([])
    if title is not None:
        plt.title(title, size=17)
#     plt.axis('off')
    plt.tight_layout()
    
plot_clustering(X, labels_true, "Original")
plt.show()

#----------------------------------------------------------------------

gmm = mixture.GaussianMixture(n_components=3, covariance_type='full').fit(X)
gmm.predict(X)
print('Srednie:\n')
print(gmm.means_)

print('\n\nKowariancje:\n')
print(gmm.covariances_)



In [ ]:
from sklearn.datasets import load_digits


mnist = load_digits(n_class=10)
x, y = mnist.data, mnist.target
print(np.unique(y, return_counts=True))

idx = np.random.choice(y.size)
fig, ax = plt.subplots(figsize=(2, 2))
ax.imshow(x[idx].reshape((8, 8)), cmap=plt.cm.gray)
ax.axis("off")
plt.title(f'digit: {y[idx]}')
plt.show()
plt.close()


### Zadanie
Korzystając z klasy ,,GaussianMixture'' wyznacz mieszaninę rozkładów normalnych dla każdej z cyfr. Wygeneruj kilka przykładowych obrazków ze znalezionego rozkładu normalnego. Możesz wykorzystać funkcję [multivariate_normal](https://numpy.org/doc/stable/reference/random/generated/numpy.random.multivariate_normal.html) do generowania sampli. 

## Estymacja gęstości

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


mu = 4
sigma = 5
n = 1000
x = np.random.normal(loc=mu_1, scale=sigma_1, size=n)

num_bins = 100
fig, ax = plt.subplots(figsize=(10, 6))
n, bins, patches = ax.hist(x, num_bins, density=True)

y = ((1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
ax.plot(bins, y, '--r')

plt.show()
plt.close()


In [ ]:
from sklearn.neighbors import KernelDensity


n, m = 1000, .3
x = np.random.randn(n)
x[int(m * n):] += 3

kde = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(x[:, None])

v = np.linspace(-5, 10, 1000)
# score_samples returns the log of the probability density
logprob = kde.score_samples(v[:, None])

num_bins = 50
fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(6, 10), sharex=True)
axs[0].hist(x, num_bins, density=True)
axs[1].fill_between(v, np.exp(logprob), alpha=0.5)
axs[1].plot(v, np.exp(logprob), "-r")
axs[1].plot(x, np.full_like(x, -0.01), '|k', markeredgewidth=1)

axs[2].hist(x, num_bins, density=True)
axs[2].fill_between(v, np.exp(logprob), alpha=0.5)
axs[2].plot(v, np.exp(logprob), "-r")

# plt.ylim(-0.02, 0.22)
plt.show()
plt.close()

## Zadanie

Wykonaj estymację gęstości zbioru cyfr (można zmniejszyć ich wymiarowość aby obliczenia trwały krócej), a następnie wygeneruj losowo kilka cyfr i je narysuj.